# Make Faiss Index from wiki_dqr

In [1]:
from typing import Any

from langchain_community.vectorstores import FAISS
import datasets
import tqdm

from dpr_embedding import CustomEmbeddings

DEVICE = "cuda:0"

In [2]:
wiki_datasets = datasets.load_dataset("wiki_dpr", "psgs_w100.nq.compressed")

/compuworks/anaconda3/envs/xais_torch23/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for wiki_dpr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wiki_dpr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/161 [00:00<?, ?it/s]

In [3]:
#replaced by 'from dpr_embedding import CustomEmbeddings'
'''
class CustomEmbeddings(Embeddings):
    def __init__(self, model_name_or_path, model_kwargs=None):
        if not model_kwargs:
            model_kwargs = {}
        self.tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(model_name_or_path)
        self.model = DPRQuestionEncoder.from_pretrained(model_name_or_path, **model_kwargs)
        self.device = model_kwargs["device_map"] if "device_map" in model_kwargs else "cpu" 
    def embed_documents(self, text):
        tmp = self.tokenizer(text, padding=True, return_tensors="pt").to(self.device)
        input_ids, token_type_ids, attention_mask = tmp["input_ids"], tmp["token_type_ids"], tmp["attention_mask"]
        res = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids).pooler_output.detach().cpu().numpy()
        #print(res.shape)
        return res
    def embed_query(self, text: str):
        """Compute query embeddings using a HuggingFace transformer model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        return self.embed_documents([text])[0]
'''

embedding = CustomEmbeddings("facebook/dpr-question_encoder-single-nq-base", model_kwargs={"device_map": DEVICE})

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
faiss = None

stride = 10000
datasets_len = len(wiki_datasets["train"])
for i in tqdm.trange(0, datasets_len, stride):
    l, r = i, min(i+stride, datasets_len)
    tmp = wiki_datasets["train"][l:r]
    texts = tmp["text"]
    embeddings = tmp["embeddings"]
    if faiss:
        faiss.add_embeddings(zip(texts, embeddings))
    else:
        faiss = FAISS.from_embeddings(zip(texts, embeddings), embedding=embedding, distance_strategy="MAX_INNER_PRODUCT")

faiss.save_local("./db/faiss")

NameError: name 'raw_datasets' is not defined

# FAISS TEST

In [10]:
# load faiss if you resume from here.
faiss = FAISS.load_local("./db/faiss", embeddings=embedding, allow_dangerous_deserialization=True)

In [39]:
retriever = faiss.as_retriever(search_kwargs={"k":4})

docs = retriever.invoke("who is the aaron?")
docs